In [111]:
import numpy as np

with open('./coef_model2.npy', 'rb') as f:
    coef = np.load(f)

coef_NKI = coef[:, 0]
coef_share = coef[:, 1]

In [112]:
import gzip
import pickle

filename = './slider_values.pkl.gz'

with gzip.open(filename, 'rb') as f:
    X_df = pickle.load(f)
    
filename = './outcomes.pkl.gz'

with gzip.open(filename, 'rb') as f:
    Y_df = pickle.load(f)

In [113]:
import ipywidgets as widgets

knowledge_init = X_df.loc['knowledge', 2019]
punctuality_init = X_df.loc['punctuality', 2019]
relevance_init = X_df.loc['relevance', 2019]
value_init = X_df.loc['value', 2019]

knowledge = widgets.FloatSlider(
    value=knowledge_init,
    min=knowledge_init - 0.1,
    max=knowledge_init + 0.1,
    step=0.01,
    description='knowledge',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

punctuality = widgets.FloatSlider(
    value=punctuality_init,
    min=punctuality_init - 0.1,
    max=punctuality_init + 0.1,
    step=0.01,
    description='punctuality',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

relevance = widgets.FloatSlider(
    value=relevance_init,
    min=relevance_init - 0.1,
    max=relevance_init + 0.1,
    step=0.01,
    description='relevance',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

value = widgets.FloatSlider(
    value=value_init,
    min=value_init - 0.1,
    max=value_init + 0.1,
    step=0.01,
    description='value',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

slider_layout = widgets.VBox([knowledge, punctuality, relevance, value])

In [114]:
# import bqplot.pyplot as plt
# import pandas as pd

# X = np.array([
# 1,
# knowledge.value,
# punctuality.value,
# relevance.value,
# value.value
# ])

# outputs = pd.DataFrame({
# 'outcomes':['NKI', 'market_share'],
# 'value':[round(X@coef_NKI, 3), round(X@coef_share, 3)]
# })

# title = f'''
# What-if\n
# original NKI = {Y_df.loc['Nöjd Kund Index (NKI) allmänheten', 2019]}, 
# original market share = {Y_df.loc['Kollektivtrafikens marknadsandel', 2019]}\n
# estimated NKI = {outputs.iloc[0,1]},
# estimated market share = {outputs.iloc[1,1]})
# '''

# fig = px.bar(outputs, x='outcomes', y='value',
#              width=500, height=300, range_y = [0, 0.8], text='value', title=title)

# <center>Example Dashboard</center>

### Original values in 2019: 
#### NKI = **0.54**, Market share = **0.32**

In [117]:
from ipywidgets import interact, HBox, VBox, interactive_output
import pandas as pd
import plotly.express as px

def plot(knowledge, punctuality, relevance, value):
    X = np.array([
    1,
    knowledge,
    punctuality,
    relevance,
    value
    ])
    
    outputs = pd.DataFrame({
    'outcomes':['NKI', 'market_share'],
    'value':[round(X@coef_NKI, 3), round(X@coef_share, 3)]
    })
    
    fig = px.bar(outputs, x='outcomes', y='value',
                width=400, height=300, range_y = [0, 0.8], text='value',
                title='Estimated outcomes')

    return fig.show()

out = widgets.interactive_output(plot, 
                                 {'knowledge': knowledge, 'punctuality': punctuality, 'relevance': relevance, 'value':value})

layout = widgets.Layout(display='flex',
                flex_flow='raw',
                align_items='center',
                width='100%')
display(HBox([VBox([knowledge, punctuality, relevance, value]), out], layout=layout))

knowledge = For most trips I make, I know how to go with the company.

punctuality = I can trust that I will arrive on time with the company.

relevance = I can use the company for most trips I make.

value (for money) = It is affordable to travel with the company.